In [1]:
from parse import Parser
from scanner import Symbol, Scanner
from names import Names
from network import Network
from devices import Devices
from monitors import Monitors
from userint import UserInterface

names = Names()

# DEVICES
SWITCH = Symbol("SWITCH", names.lookup(["SWITCH"])[0], 0, 0)
swtich1 = Symbol("switch1", names.lookup(["switch1"])[0], 0, 0)
zero = Symbol("0", names.lookup(["0"])[0], 0, 0)
swtich2 = Symbol("switch2", names.lookup(["switch2"])[0], 0, 0)
AND = Symbol("AND", names.lookup(["AND"])[0], 0, 0)
and1 = Symbol("and1", names.lookup(["and1"])[0], 0, 0)
OR = Symbol("OR", names.lookup(["OR"])[0], 0, 0)
or1 = Symbol("or1", names.lookup(["or1"])[0], 0, 0)
two = Symbol("2", names.lookup(["2"])[0], 0, 0)
NAND = Symbol("NAND", names.lookup(["NAND"])[0], 0, 0)
nand1 = Symbol("nand1", names.lookup(["nand1"])[0], 0, 0)
DTYPE = Symbol("DTYPE", names.lookup(["DTYPE"])[0], 0, 0)
dtype1 = Symbol("dtype1", names.lookup(["dtype1"])[0], 0, 0)
CLOCK = Symbol("CLOCK", names.lookup(["CLOCK"])[0], 0, 0)
clock1 = Symbol("clock1", names.lookup(["clock1"])[0], 0, 0)
one = Symbol("1", names.lookup(["1"])[0], 0, 0)

# CONNECT
arrow = Symbol(">", names.lookup([">"])[0], 0, 0)
I1 = Symbol("I1", names.lookup(["I1"])[0], 0, 0)
dot = Symbol(".", names.lookup(["."])[0], 0, 0)
I2 = Symbol("I2", names.lookup(["I2"])[0], 0, 0)
Q = Symbol("Q", names.lookup(["Q"])[0], 0, 0)
Q_BAR = Symbol("Q_BAR", names.lookup(["Q_BAR"])[0], 0, 0)
DATA = Symbol("DATA", names.lookup(["DATA"])[0], 0, 0)
SET = Symbol("SET", names.lookup(["SET"])[0], 0, 0)
CLK = Symbol("CLK", names.lookup(["CLK"])[0], 0, 0)
CLEAR = Symbol("CLEAR", names.lookup(["CLEAR"])[0], 0, 0)


dict = {"DEVICES": [[SWITCH, swtich1, zero], [SWITCH, swtich2, zero],
                    [AND, and1, two], [OR, or1, two], [NAND, nand1, two],
                    [DTYPE, dtype1], [CLOCK, clock1, one]],
        "CONNECTIONS": [[swtich1, arrow, and1, dot, I1],
                    [swtich1, arrow, or1, dot, I1],
                    [swtich2, arrow, and1, dot, I2],
                    [swtich1, arrow, or1, dot, I2],
                    [and1, arrow, nand1, dot, I1],
                    [or1, arrow, nand1, dot, I2],
                    [clock1, arrow, dtype1, dot, CLK],
                    [swtich1, arrow, dtype1, dot, DATA],
                    [nand1, arrow, dtype1, dot, SET],   
                    [or1, arrow, dtype1, dot, CLEAR]],

        "MONITORS": [[and1], [or1], [nand1]]}

devices = Devices(names=names)
network = Network(names=names, devices=devices)
monitors = Monitors(names=names, devices=devices, network=network)
scanner = Scanner(None, names=names)
parser = Parser(names=names, devices=devices, network=network, monitors=monitors, scanner=scanner)
parser.build_network(dict)
ui = UserInterface(names=names, devices=devices, network=network, monitors=monitors)

In [2]:
for device in devices.devices_list:
    id = device.device_id
    name = names.get_name_string(id)
    type = device.device_kind
    print(f"{id} {name} {type} {device.inputs} {device.outputs}")

1 switch1 0 {} {None: 0}
3 switch2 0 {} {None: 0}
5 and1 4 {17: (1, None), 19: (3, None)} {None: 0}
7 or1 6 {17: (1, None), 19: (1, None)} {None: 0}
10 nand1 9 {17: (5, None), 19: (7, None)} {None: 0}
12 dtype1 11 {24: (14, None), 23: (10, None), 25: (7, None), 22: (1, None)} {20: 0, 28: 0}
14 clock1 13 {} {None: 1}


In [3]:
switches = ui.devices.find_devices(devices.SWITCH)

In [4]:
# Getting the switch names and setting them

In [5]:
switches_dict = {}
for switch_id in switches:
    switch_name = names.get_name_string(switch_id)
    switches_dict[switch_name] = switch_id

In [6]:
# This is how to get the names of the switches and their ids
switches_dict

{'switch1': 1, 'switch2': 3}

In [5]:
switch_1_id = switches[0]
switch_2_id = switches[1]

switch_1_name = names.get_name_string(switch_1_id)
switch_2_name = names.get_name_string(switch_2_id)

In [6]:
switch_1_name, switch_2_name

('switch1', 'switch2')

In [7]:
ui.devices.set_switch(switch_1_id, 1)

True

In [8]:
ui.devices.set_switch(switch_2_id, 1)

True

In [9]:
# Continue running cycles for certain amount of time

In [10]:
ui.run_network(5)

and1 : -----
or1  : -----
nand1: _____


True

In [11]:
# When run pressed - reset network and devices
ui.devices.cold_startup()
ui.monitors.reset_monitors()

In [12]:
ui.run_network(0)

and1 : 
or1  : 
nand1: 


True

In [13]:
ui.run_network(5)

and1 : -----
or1  : -----
nand1: _____


True

In [14]:
# Add monitor

ui.monitors.make_monitor(switch_1_id, None, ui.cycles_completed)


12

In [15]:
ui.run_network(5)

and1   : ----------
or1    : ----------
nand1  : __________
switch1: -----


True

In [16]:
ui.monitors.make_monitor(switch_2_id, None, ui.cycles_completed)

12

In [17]:
ui.run_network(5)

and1   : ---------------
or1    : ---------------
nand1  : _______________
switch1: ----------
switch2: -----


True

In [18]:
ui.devices.set_switch(switch_1_id, 0)

True

In [19]:
ui.run_network(5)

and1   : ---------------_____
or1    : ---------------_____
nand1  : _______________-----
switch1: ----------_____
switch2: ----------


True

In [20]:
# Remove monitor

ui.monitors.remove_monitor(switch_1_id, None)

True

In [21]:
ui.run_network(5)

and1   : ---------------__________
or1    : ---------------__________
nand1  : _______________----------
switch2: ---------------


True

In [22]:
ui.devices.cold_startup()
ui.monitors.reset_monitors()

In [23]:
ui.monitors.monitors_dictionary

OrderedDict([((5, None), []),
             ((7, None), []),
             ((10, None), []),
             ((3, None), [])])

In [24]:
ui.run_network(5)
ui.monitors.monitors_dictionary

and1   : _____
or1    : _____
nand1  : -----
switch2: -----


OrderedDict([((5, None), [0, 0, 0, 0, 0]),
             ((7, None), [0, 0, 0, 0, 0]),
             ((10, None), [1, 1, 1, 1, 1]),
             ((3, None), [1, 1, 1, 1, 1])])

In [25]:
ui.devices.set_switch(switch_2_id, 0)
ui.run_network(5)
ui.monitors.monitors_dictionary

and1   : __________
or1    : __________
nand1  : ----------
switch2: -----_____


OrderedDict([((5, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((7, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((10, None), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
             ((3, None), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0])])

In [26]:
# Lets add switch 1 to the monitor again
ui.monitors.make_monitor(switch_1_id, None, ui.cycles_completed)
ui.monitors.monitors_dictionary

OrderedDict([((5, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((7, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((10, None), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
             ((3, None), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]),
             ((1, None), [])])

In [27]:
ui.run_network(5)
ui.monitors.monitors_dictionary

and1   : _______________
or1    : _______________
nand1  : ---------------
switch2: -----__________
switch1: _____


OrderedDict([((5, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((7, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((10, None), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
             ((3, None), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
             ((1, None), [0, 0, 0, 0, 0])])

In [28]:
# So when a monitor is added, it seems that it only shows values from where it
# was added onwards but with the indexing making it seem like it was there from
# the start, will need to modify the graphs to deal with this

In [30]:
# Want to get a list of the outputs of all the devices

In [51]:
outputs = {}
for device in devices.devices_list:
    device_id = device.device_id
    device_name = names.get_name_string(device_id)
    device_outputs = device.outputs
    print(f"{device_id} {device_name} {device_outputs}")
    for id in device_outputs:
        if id is None:
            outputs[f"{device_name}"] = (device_id, None)
        else:
            pin_name = names.get_name_string(id)
            outputs[f"{device_name}.{pin_name}"] = (device_id, pin_name) 

1 switch1 {None: 0}
3 switch2 {None: 0}
5 and1 {None: 0}
7 or1 {None: 0}
10 nand1 {None: 0}
12 dtype1 {20: 0, 28: 0}
14 clock1 {None: 1}


In [52]:
# Getting the outputs for the monitors
outputs

{'switch1': (1, None),
 'switch2': (3, None),
 'and1': (5, None),
 'or1': (7, None),
 'nand1': (10, None),
 'dtype1.Q': (12, 'Q'),
 'dtype1.QBAR': (12, 'QBAR'),
 'clock1': (14, None)}